In [ ]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

# Provided data
months = list(range(1, 145)) # months from M1 to M144
production = np.array([
    1863, 1614, 2570, 1685, 2101, 1811, 2457, 2171, 2134, 2502, 2358, 2399,
    2048, 2523, 2086, 2391, 2150, 2340, 3129, 2277, 2964, 2997, 2747, 2862,
    3405, 2677, 2749, 2755, 2963, 3161, 3623, 2768, 3141, 3439, 3601, 3531,
    3477, 3376, 4027, 3175, 3274, 3334, 3964, 3649, 3502, 3688, 3657, 4422,
    4197, 4441, 4736, 4521, 4485, 4644, 5036, 4876, 4789, 4544, 4975, 5211,
    4880, 4933, 5079, 5339, 5232, 5520, 5714, 5260, 6110, 5334, 5988, 6235,
    6365, 6266, 6345, 6118, 6497, 6278, 6638, 6590, 6271, 7246, 6584, 6594,
    7092, 7326, 7409, 7976, 7959, 8012, 8195, 8008, 8313, 7791, 8368, 8933,
    8756, 8613, 8705, 9098, 8769, 9544, 9050, 9186, 10012, 9685, 9966, 10048,
    10244, 10740, 10318, 10393, 10986, 10635, 10731, 11749, 11849, 12123,
    12274, 11666, 11960, 12629, 12915, 13051, 13387, 13309, 13732, 13162,
    13644, 13808, 14101, 13992, 15191, 15018, 14917, 15046, 15556, 15893,
    16388, 16782, 16716, 17033, 16896, 17689
])

# Generating x values representing the month indices
months_array = np.arange(1, len(production) + 1).reshape(-1, 1)

# Degree of the polynomial
poly_degree = 4

# Creating polynomial features
poly_features = PolynomialFeatures(degree=poly_degree)
X_poly = poly_features.fit_transform(months_array)

# Define the Newton-Raphson function
def newton_raphson_method(X, y, max_iterations=100, tolerance=1e-6):
    num_samples, num_features = X.shape
    theta = np.zeros(num_features)
    
    for _ in range(max_iterations):
        predictions = X.dot(theta)
        residuals = predictions - y
        gradient = X.T.dot(residuals) / num_samples
        hessian = X.T.dot(X) / num_samples
        delta_theta = np.linalg.solve(hessian, gradient)
        theta -= delta_theta
        
        if np.linalg.norm(delta_theta) < tolerance:
            break
    
    return theta

# Apply Newton-Raphson method to find the parameters
theta_values = newton_raphson_method(X_poly, production)

# Polynomial function based on theta
def polynomial_model(theta, x):
    return sum(theta[i] * (x ** i) for i in range(len(theta)))

# Find the root where the production exceeds 25,000 bags
def find_threshold_root(theta, target, start=1, end=25000, tolerance=1e-6):
    while start <= end:
        mid_point = (start + end) // 2
        if polynomial_model(theta, mid_point) > target:
            end = mid_point - 1
        else:
            start = mid_point + 1
    return round(start - 1 + (target - polynomial_model(theta, start - 1)) / (polynomial_model(theta, start) - polynomial_model(theta, start - 1)), 3)

# Target production threshold
threshold = 25000

# Find the month where the production exceeds the threshold
month_threshold = find_threshold_root(theta_values, threshold)

# EGIER needs to start building the warehouse 13 months before this month
start_building_month = month_threshold - 13

# Output the result
print(f"EGIER needs to start building the new warehouse in {start_building_month:.3f} month.")
